In [ ]:
import numpy as np
import pandas as pd
import shapely.geometry as geom
import data_utils as utils
import time 
import reverse_geocoder as rg
import importlib
import osmnx as ox
import re
import math

In [41]:
data = pd.read_csv("joined_aadt.csv")
nodesdf = pd.read_csv("accident_node_data.csv")

/home/asifmallik/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [52]:
display(data.head())
print(len(data))
display(nodesdf.head())

,Road Name,Beginning Description,Ending Description,Count_2014,Count_2015,Count_2016,Count_2017,Count_2018,Station ID,County,...,TruckPercentSourceFieldName,DOT_RegionName,DOT_RegionNumber,DOT_CountyNumber,MorningHighestValue,AfternoonHighestValue,EveningHighestValue,RCSTAYearDirection,SHAPE_Length,geometry
0,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,NaN,534.0,653.0,719.0,"010009, 2019, NB",811.603299,MULTILINESTRING ((-73.82738679036918 40.886117...
1,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,NaN,1107.0,1455.0,1557.0,"010009, 2019, CT",811.603299,MULTILINESTRING ((-73.82738679036918 40.886117...
2,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,NaN,618.0,802.0,852.0,"010009, 2019, SB",811.603299,MULTILINESTRING ((-73.82738679036918 40.886117...
3,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,NaN,534.0,653.0,719.0,"010009, 2019, NB",863.285819,MULTILINESTRING ((-73.82794249915982 40.885920...
4,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,NaN,1107.0,1455.0,1557.0,"010009, 2019, CT",863.285819,MULTILINESTRING ((-73.82794249915982 40.885920...


25357


,datetime,longitude,latitude,node
0,2012-07-01 00:05:00,-73.813916,40.697753,42829106
1,2012-07-01 00:05:00,-73.997387,40.762127,42430680
2,2012-07-01 00:10:00,-73.923840,40.733610,597311311
3,2012-07-01 00:10:00,-73.972745,40.588868,42519347
4,2012-07-01 00:20:00,-73.983048,40.677406,42486357


              datetime  longitude   latitude      node
0  2012-07-01 00:05:00 -73.813916  40.697753  42829106
1  2012-07-01 00:05:00 -73.997387  40.762127  42430680


# Selecting points from manhattan

Here we will select only the sections that are relevant to us. Namely, those pertaining to manhattan. First I select the nodes that are in manhattan

In [17]:
import geopandas as gpd

In [18]:
boro = gpd.read_file('borough_boundaries.geojson')
manhattan_poly = boro['geometry'].iloc[2]

In [19]:
boro.head()

,boro_code,boro_name,shape_area,shape_leng,geometry
0,2,Bronx,1186612670.99,462958.559564,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ..."
1,5,Staten Island,1623757282.78,325956.009,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ..."
2,1,Manhattan,636602659.031,361212.477841,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ..."
3,3,Brooklyn,1934091267.17,729593.057284,"MULTIPOLYGON (((-73.86706 40.58209, -73.86769 ..."
4,4,Queens,3044042343.6,899490.723339,"MULTIPOLYGON (((-73.82645 40.59053, -73.82642 ..."


In [20]:
# Select nodes that are located in manhattan
points = nodesdf[['longitude','latitude']].to_numpy()
coordinates = [geom.Point(point) for point in points]
inside_manhattan = [point.within(manhattan_poly) for point in coordinates]


KeyboardInterrupt: 

In [ ]:
manhattan_accidents = nodesdf[inside_manhattan]
manhattan_accidents.to_csv("manhattan_accidents_node_data.csv")

In [22]:
manhattan_accidents.head()

,Unnamed: 0,datetime,longitude,latitude,node
0,1,2012-07-01 00:05:00,-73.997387,40.762127,42430680
1,8,2012-07-01 00:30:00,-74.005887,40.714678,42428402
2,10,2012-07-01 00:37:00,-73.974889,40.755924,42448811
3,12,2012-07-01 00:45:00,-73.964503,40.758976,42423564
4,19,2012-07-01 01:16:00,-74.003976,40.732914,42431797


Now, I will choose the points per each of the nodes inside of manhattan. 

In [23]:
unique_nodes, index_unique = np.unique(manhattan_accidents[['node']].to_numpy(), return_index = True)


I will use the graphml file containing latitudes and longitudes to obtain this information

In [24]:
graph_file = 'newyork.graphml'
graph = ox.io.load_graphml(graph_file)

In [25]:
# Latitude and longitude coordinates of the nodes
unique_x = []
unique_y = []
for node in unique_nodes: 
    unique_x.append(graph.nodes[node]['x'])
    unique_y.append(graph.nodes[node]['y'])
unique_x = np.expand_dims(np.array(unique_x), 1)
unique_y = np.expand_dims(np.array(unique_y), 1)
unique_points = np.concatenate((unique_x, unique_y), axis = 1)



Now we will repeat something very similar but with the road sections. We will select only those that are located in Manhattan

In [26]:
manhattan_roads = data[data['County'].apply(lambda x: x == 'New York')]
manhattan_roads.to_csv('manhattan_road_segments_aadt.csv')

# Extracting line segments

In [27]:
p = re.compile(r'[-+]?[0-9]*\.?[0-9]+')
xy = manhattan_roads['geometry'].apply(lambda x: p.findall(x))

In [28]:
def transform(x): 
    points = []
    j = 0
    point = []
    for i in range(len(x)):
        if j == 0:
            point = [float(x[i])]
            j += 1
        elif j == 1: 
            point.append(float(x[i]))
            points.append(point)
            j = 0
    return np.array(points)
    

In [29]:
def find_min_segment(point,segments,cutoff=True): 
    """
    returns index of closest segment. 
    Assumes point is np.array([x,y])
    Segments is an array of road semgments each item in 
    the list represets one road. 
    """
    def return_min(segment):
        return np.nanmin(utils.lineseg_dists(point,segment[:-1],segment[1:]))
    
    # We are looking for all roads closer than 0.01 km. 
    # A unit in long-lat coordinates is equal to about 111 km
    distances = np.array([return_min(segment) for segment in segments])
    if cutoff: 
        return (distances < (0.01/111)).nonzero()[0]
    else:
        return np.nanargmin(distances)


In [30]:
# This is to get a represenation of the coordinates the segments as an array of arrays
# Each array represents a road segment. 

segments = xy.apply(transform)
segments = segments.to_numpy()

In [90]:
minim = []
start = time.time()
nodes_roads_data = {}

cols = []
for n in range(4, 10):
    cols.append('Count_201' + str(n))

data['Count_mean'] = data[cols].mean(axis=1)
cols.append('Count_mean')
display(data.head())
for col in cols:
    nodes_roads_data[col] = []   


for i in range(len(unique_points)):
    if i % 100 == 0:
        print(i)
    road_ids = list(find_min_segment(unique_points[i],segments, cutoff=True))
    for col in cols:
        max_AADT = data.iloc[road_ids].dropna(subset=[col])[col].max()
        nodes_roads_data[col].append(max_AADT)
        
    minim.append(",".join(map(str, road_ids)))
print(time.time() - start)    


,Road Name,Beginning Description,Ending Description,Count_2014,Count_2015,Count_2016,Count_2017,Count_2018,Station ID,County,...,DOT_RegionName,DOT_RegionNumber,DOT_CountyNumber,MorningHighestValue,AfternoonHighestValue,EveningHighestValue,RCSTAYearDirection,SHAPE_Length,geometry,Count_mean
0,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,534.0,653.0,719.0,"010009, 2019, NB",811.603299,MULTILINESTRING ((-73.82738679036918 40.886117...,21950.0
1,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,1107.0,1455.0,1557.0,"010009, 2019, CT",811.603299,MULTILINESTRING ((-73.82738679036918 40.886117...,21950.0
2,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,618.0,802.0,852.0,"010009, 2019, SB",811.603299,MULTILINESTRING ((-73.82738679036918 40.886117...,21950.0
3,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,534.0,653.0,719.0,"010009, 2019, NB",863.285819,MULTILINESTRING ((-73.82794249915982 40.885920...,21950.0
4,BOSTON RD,RT 22 PROVOST AVE,Bronx/West Co Line,23301.0,22471.0,NaN,NaN,21014.0,10009,Bronx,...,NaN,NaN,NaN,1107.0,1455.0,1557.0,"010009, 2019, CT",863.285819,MULTILINESTRING ((-73.82794249915982 40.885920...,21950.0


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
313.6588475704193


In [74]:
num = 0
for i in range(len(minim)):
    if len(minim[i]) == 0:
        num += 1
print(num)    

1278


In [87]:
nodes_roads_data['nodes'] = unique_nodes
nodes_roads_data['roads'] = minim

intersections = pd.DataFrame(data=nodes_roads_data)
intersections = intersections.dropna(subset=cols, how="all")

In [88]:
intersections.to_csv("nodes_roads.csv")